In [1]:
import os
import random
import gc
import argparse
import numpy as np
import h5py as h5
import torch
import torch.nn.functional as F
import wandb

from models.vae import VAE 
from losses.cyl_ptpz_mae import CylPtPzMAE

from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#setting a seed like in vae_legacy
def set_seed(seed=123):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
#calculated disco loss based on paper
def disco_loss(z1, z2):
    #center around 0
    x = z1 - z1.mean(0)
    y = z2 - z2.mean(0)
    #build euclidean distance matrices
    a = torch.cdist(x, x)
    b = torch.cdist(y, y)
    #doublce center distance matrices
    A = a - a.mean(0) - a.mean(1, keepdim=True) + a.mean()
    B = b - b.mean(0) - b.mean(1, keepdim=True) + b.mean()
    #find covariance
    dcov = (A * B).mean()
    #find variance and take square root (in paper it is not squared)
    dvar_x = (A * A).mean().sqrt()
    dvar_y = (B * B).mean().sqrt()
    #return paper formula for disco
    return dcov / (dvar_x * dvar_y + 1e-10)

In [4]:
def distance_corr(var_1,var_2,normedweight,power=1):
    """var_1: First variable to decorrelate (eg mass)
    var_2: Second variable to decorrelate (eg classifier output)
    normedweight: Per-example weight. Sum of weights should add up to N (where N is the number of examples)
    power: Exponent used in calculating the distance correlation
    
    va1_1, var_2 and normedweight should all be 1D torch tensors with the same number of entries
    
    Usage: Add to your loss function. total_loss = BCE_loss + lambda * distance_corr
    """ 
    
    xx = var_1.view(-1, 1).repeat(1, len(var_1)).view(len(var_1),len(var_1))
    yy = var_1.repeat(len(var_1),1).view(len(var_1),len(var_1))
    amat = (xx-yy).abs()

    xx = var_2.view(-1, 1).repeat(1, len(var_2)).view(len(var_2),len(var_2))
    yy = var_2.repeat(len(var_2),1).view(len(var_2),len(var_2))
    bmat = (xx-yy).abs()

    amatavg = torch.mean(amat*normedweight,dim=1)
    Amat=amat-amatavg.repeat(len(var_1),1).view(len(var_1),len(var_1))\
        -amatavg.view(-1, 1).repeat(1, len(var_1)).view(len(var_1),len(var_1))\
        +torch.mean(amatavg*normedweight)

    bmatavg = torch.mean(bmat*normedweight,dim=1)
    Bmat=bmat-bmatavg.repeat(len(var_2),1).view(len(var_2),len(var_2))\
        -bmatavg.view(-1, 1).repeat(1, len(var_2)).view(len(var_2),len(var_2))\
        +torch.mean(bmatavg*normedweight)

    ABavg = torch.mean(Amat*Bmat*normedweight,dim=1)
    AAavg = torch.mean(Amat*Amat*normedweight,dim=1)
    BBavg = torch.mean(Bmat*Bmat*normedweight,dim=1)

    if(power==1):
        dCorr=(torch.mean(ABavg*normedweight))/torch.sqrt((torch.mean(AAavg*normedweight)*torch.mean(BBavg*normedweight)))
    elif(power==2):
        dCorr=(torch.mean(ABavg*normedweight))**2/(torch.mean(AAavg*normedweight)*torch.mean(BBavg*normedweight))
    else:
        dCorr=((torch.mean(ABavg*normedweight))/torch.sqrt((torch.mean(AAavg*normedweight)*torch.mean(BBavg*normedweight))))**power
    
    return dCorr

In [5]:
#calculates anomaly score like in vae legacy
def distance_pt(model_vae, data_np, device):
    x = torch.tensor(data_np, dtype=torch.float32, device=device)
    z_mean, z_logvar, _ = model_vae.encoder(x)
    score = torch.sum(z_mean**2, dim=1)
    return score.detach().cpu().numpy()

In [6]:
#function to make 2D histograms
def make_2D_hist(x, y, xlabel, ylabel, title, wandb_key, bins=40):
    fig = plt.figure(figsize=(5,4))
    plt.hist2d(x, y, bins=bins)
    plt.colorbar(label='Counts')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    wandb.log({wandb_key: wandb.Image(fig)})
    plt.close(fig)

In [7]:
#prints h5 tree to examine
def print_h5_tree(h, prefix=""):
    for k in h.keys():
        item = h[k]
        if hasattr(item, 'keys'):
            print(prefix + f"[GROUP] {k}")
            print_h5_tree(item, prefix + "  ")
        else:
            try:
                print(prefix + f"{k}: shape={item.shape}, dtype={item.dtype}")
            except Exception:
                print(prefix + f"{k}: <dataset>")

In [9]:
def run(config):
    #set seed
    seed = 123
    set_seed(seed)

    #move to gpu if avail
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print(f"Using: {device}")

    #login to wandb
    print("Logging in to wandb...")
    wandb.login(key="24d1d60ce26563c74d290d7b487cb104fc251271")
    wandb.init(project="Double Disco Axo Training",
               settings=wandb.Settings(_disable_stats=True),
               config=config)
    run_name = wandb.run.name
    print(f"Run name: {run_name}")

    #scaling
    beta = float(config['beta'])
    alpha = float(config['alpha'])
    vae_lr = float(config['vae_lr'])

    #load data
    print("Loading dataset...")
    fpath = '/axovol/training/v5/conditionsupdate_apr25.h5'
    with h5.File(fpath, 'r') as f:
        root = f['data'] if 'data' in f else f

        #print h5 tree to view
        print("Printing h5 tree...")
        print_h5_tree(root)
    
        x_train = root['Background_data']['Train']['DATA'][:]
        x_test = root['Background_data']['Test']['DATA'][:]
        print(f"Train shape: {x_train.shape}, Test shape: {x_test.shape}")
    
        #flatten per event
        x_train_bkg = x_train.reshape(x_train.shape[0], -1)
        x_test_bkg  = x_test.reshape(x_test.shape[0], -1)
    
        scale = root['Normalisation']['norm_scale'][:]
        bias = root['Normalisation']['norm_bias'][:]
    
        l1_bits_bkg_test = root['Background_data']['Test']['L1bits'][:]
    
        #load signal data
        SIGNAL_NAMES = list(root['Signal_data'].keys())
        signal_data_dict = {}
        signal_l1_dict = {}
        for sname in SIGNAL_NAMES:
            x_sig = root['Signal_data'][sname]['DATA'][:]
            x_sig = x_sig.reshape(x_sig.shape[0], -1)
            l1_bits = root['Signal_data'][sname]['L1bits'][:]
            signal_data_dict[sname] = x_sig
            signal_l1_dict[sname] = l1_bits

    print("Data finished loading.")
    
    dataset = x_train_bkg
    dataset_test = x_test_bkg

    #should be 57
    features = dataset.shape[1]


    print("Building mask...")
    # same mask from vae legacy
    mask_dict = {
        "MET":[True],
        "EGAMMA":[True,True,True,True,False,False,False,False,False,False,False,False],
        "MUON":[True,True,True,True,False,False,False,False],
        "JET":[True,True,True,True,True,True,True,True,True,True,False,False]
    }
    
    #build cyl_ptpz_mae mask (input scales and biases)
    reco_loss_fn = CylPtPzMAE(scale, bias).to(device)
    print("Mask is ready.")


    #vae config
    latent_dim = int(config['vae_latent'])
    enc_nodes = list(config['vae_nodes'])
    dec_nodes = [24, 32, 64, 128, features] 

    vae_cfg = {
        "features": features,
        "latent_dim": latent_dim,
        "encoder_config": {"nodes": enc_nodes},
        "decoder_config": {"nodes": dec_nodes},
        "alpha": alpha,
        "beta":  beta,
    }

    #put vae on device/init
    vae_1 = VAE(vae_cfg).to(device)
    vae_2 = VAE(vae_cfg).to(device)
    print("VAEs are ready.")

    #optimizer (adam)
    optimizer = torch.optim.Adam(list(vae_1.parameters()) + list(vae_2.parameters()), lr=vae_lr)

    #cosine restarts
    warmup_epochs = 10
    cos = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=32, T_mult=2, eta_min=0.0)

    #sets learning rate
    def set_lr(lr):
        for g in optimizer.param_groups:
            g['lr'] = lr

    #hyperparameters
    Epochs_VAE = 200
    Batch_size = 16384

    #get disco param from config, else set to 1
    lambda_disco = float(config.get("lambda_disco", 1))

    print("Moving data to device...")
    X = torch.tensor(dataset, dtype=torch.float32, device=device)
    print("Data on device.")

    #training loop
    print("Starting the training loop!")
    N = X.size(0)
    for epoch in range(Epochs_VAE):

        #init lists for 2D plotting
        vae1_total_loss = []
        vae2_total_loss = []
        vae1_kl_loss = []
        vae2_kl_loss = []
        vae1_reco_loss = []
        vae2_reco_loss = []
        
        #cosine warmup step (sets the learning rate based on cosine schedule)
        if epoch < warmup_epochs:
            lr = vae_lr * (epoch + 1) / warmup_epochs
            set_lr(lr)
        else:
            cos.step(epoch - warmup_epochs)

        #shuffles data incides for each epoch
        perm = torch.randperm(N, device=device)

        #init losses
        total_loss = total_reco1 = total_reco2 = total_kl1 = total_kl2 = total_disco = 0.0

        #loops over dataset in steps of batch sizze
        for i in range(0, N, Batch_size):
            #picks indices for current batch
            idx = perm[i:i+Batch_size]

            #selects batch of samples from data set X
            xb = X[idx]

            #vae 1
            recon1, mu1, logvar1, z1 = vae_1(xb)
            
            #vae 2
            recon2, mu2, logvar2, z2 = vae_2(xb)

            #get reco loss from custom func
            reco1_per = reco_loss_fn(recon1, xb)
            reco2_per = reco_loss_fn(recon2, xb)

            #get kl div per sample
            kl1_per = VAE.kl_divergence(mu1, logvar1)
            kl2_per = VAE.kl_divergence(mu2, logvar2)

            tot1_per = reco1_per + kl1_per
            tot2_per = reco2_per + kl2_per

            #from paper code weight
            B = xb.shape[0]
            w = torch.ones(B, device=tot1_per.device, dtype=tot1_per.dtype)

            #disco loss (ask Melissa about since using mu instead of z)
            #disco = disco_loss(mu1, mu2)
            disco = distance_corr((tot1_per), (tot2_per), w, power=1)

            #same scaling from vae legacy
            reco1 = vae_1.reco_scale*reco1_per.mean()
            reco2 = vae_2.reco_scale*reco2_per.mean()
            kl1 = vae_1.kl_scale*kl1_per.mean()
            kl2 = vae_2.kl_scale*kl2_per.mean() 

            #calc total loss (scalar)
            loss = (reco1 + kl1) + (reco2 + kl2) + lambda_disco * disco

            #zero grads
            optimizer.zero_grad(set_to_none=True)

            #backward pass
            loss.backward()

            #do some gradient clipping
            torch.nn.utils.clip_grad_norm_(list(vae_1.parameters()) + list(vae_2.parameters()), max_norm=5.0)
            optimizer.step()

            #add losses to list for wandb plotting
            total_loss+=loss.item()
            total_reco1+=reco1.item()
            total_reco2+=reco2.item()
            total_kl1+=kl1.item()
            total_kl2+=kl2.item()
            total_disco+=disco.item()

            #add loss to list for plotting
            vae1_total_loss.append((reco1 + kl1).item())
            vae2_total_loss.append((reco2 + kl2).item())
            vae1_kl_loss.append(kl1.item())
            vae2_kl_loss.append(kl2.item())
            vae1_reco_loss.append(reco1.item())
            vae2_reco_loss.append(reco2.item())

        print(f"[EPOCH {epoch}/{Epochs_VAE}] "
          f"Loss={total_loss:.4f} "
          f"Reco1={total_reco1:.4f} Reco2={total_reco2:.4f} "
          f"KL1={total_kl1:.4f} KL2={total_kl2:.4f} "
          f"DisCo={total_disco:.4f}")

        #log in wandb
        wandb.log({
            "EpochVae": epoch,
            "TotalLossVae": total_loss,
            "RecoLossVae1": total_reco1,
            "RecoLossVae2": total_reco2,
            "KLLossVae1": total_kl1,
            "KLLossVae2": total_kl2,
            "DisCoLoss": total_disco,
        })

        #convert to np array for plot
        vae1_total_np = np.array(vae1_total_loss)
        vae2_total_np = np.array(vae2_total_loss)
        vae1_kl_np = np.array(vae1_kl_loss)
        vae2_kl_np = np.array(vae2_kl_loss)
        vae1_reco_np = np.array(vae1_reco_loss)
        vae2_reco_np = np.array(vae2_reco_loss)

        make_2D_hist(vae1_total_np, vae2_total_np,
                    "Total Loss (VAE1)", "Total Loss (VAE2)",
                    f"Epoch {epoch}: Total VAE1 vs Total VAE2",
                    wandb_key="Hists2D/Total_VAE1_vs_Total_VAE2")

        make_2D_hist(vae1_kl_np, vae2_kl_np,
                    "KL Loss (VAE1)", "KL Loss (VAE2)",
                    f"Epoch {epoch}: KL VAE1 vs KL VAE2",
                    wandb_key="Hists2D/KL_VAE1_vs_KL_VAE2")

        make_2D_hist(vae1_reco_np, vae2_reco_np,
                    "Reco Loss (VAE1)", "Reco Loss (VAE2)",
                    f"Epoch {epoch}: Reco VAE1 vs Reco VAE2",
                    wandb_key="Hists2D/Reco_VAE1_vs_Reco_VAE2")

    

    print("Finished training.")

    #save models
    torch.save(vae_1.state_dict(), "vae1_trained.pth")
    torch.save(vae_2.state_dict(), "vae2_trained.pth")




In [10]:
config = {
    'vae_lr': 1e-4,
    'beta': 0.5,
    'alpha': 0.5,
    'vae_latent': 8,
    'vae_nodes': [28, 14],
    'lambda_disco': 10000.0
}

run(config)

Using: cuda:0
Logging in to wandb...


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/jovyan/.netrc
wandb: Currently logged in as: escheuller (escheuller-uc-san-diego) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Run name: decent-breeze-137
Loading dataset...
Printing h5 tree...
[GROUP] Background_data
  [GROUP] Test
    DATA: shape=(4511092, 19, 3), dtype=float64
    ET: shape=(4511092,), dtype=float16
    HT: shape=(4511092,), dtype=float16
    L1bits: shape=(4511092,), dtype=bool
    PU: shape=(4511092, 7), dtype=float64
  [GROUP] Train
    DATA: shape=(1999965, 19, 3), dtype=float64
    ET: shape=(1999965,), dtype=float16
    HT: shape=(1999965,), dtype=float16
    L1bits: shape=(1999965,), dtype=bool
    PU: shape=(1999965, 7), dtype=float64
[GROUP] Normalisation
  norm_bias: shape=(19, 3), dtype=float64
  norm_scale: shape=(19, 3), dtype=float64
[GROUP] Signal_data
  [GROUP] GluGluHToBB_M-125
    DATA: shape=(179997, 19, 3), dtype=float64
    ET: shape=(179997,), dtype=float16
    HT: shape=(179997,), dtype=float16
    L1bits: shape=(179997,), dtype=bool
    PU: shape=(179997,), dtype=float64
  [GROUP] GluGluHToGG_M-125
    DATA: shape=(194433, 19, 3), dtype=float64
    ET: shape=(194433,